In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, LimitOrder, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

# Imports 
from utils import Vars, prepare_to_sow, place_orders, quick_pf
from loguru import logger

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

In [ ]:
df = prepare_to_sow(MARKET, 
                    build_from_scratch=False,
                    save_sow=False)

In [ ]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]
len(cos)

In [ ]:
df[OPT_COLS]

In [ ]:
PORT

In [ ]:
from utils import get_order_pf
import asyncio
with IB().connect(port=PORT) as ib:
    x = asyncio.run(get_order_pf(PORT))